In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
print(os.getcwd())
# Change directory to the root of the repository
os.chdir('..')
print(os.getcwd())

/Users/marcel/git/PhiFlow/notebooks
/Users/marcel/git/PhiFlow


In [2]:
from phi.jax.flow import *

In [3]:
import point_cloud_utils as pcu


In [4]:
# Download meshes using PyTorch Geometric
from pytorch3d.datasets.shapenet import ShapeNetCore

# Create a directory to store the data
shapenet_dataset = ShapeNetCore(data_dir='data/ShapeNetCore.v2', synsets=['watercraft'], version=2, load_textures=False)
print(list(shapenet_dataset.synset_dict.values()))

/opt/homebrew/Caskroom/miniforge/base/envs/differentiable-coupling/lib/python3.10/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:114: UserWarning: The following categories are included in ShapeNetCore ver.2's official mapping but not found in the dataset location data/ShapeNetCore.v2: 03642806, 03759954, 02818832, 04099429, 04225987, 02871439, 04090263, 03948459, 03211117, 03710193, 02747177, 02843684, 03207941, 04330267, 02958343, 03593526, 02773838, 03928116, 03085013, 03691459, 03046257, 02924116, 03513137, 03337140, 04256520, 03636649, 03790512, 02828884, 03938244, 04460130, 03325088, 02691156, 02808440, 03797390, 02946921, 02992529, 02933112, 04379243, 02954340, 04468005, 03761084, 02880940, 03991062, 02876657, 04554684, 03261776, 04074963, 02801938, 02942699, 03467517, 03001627, 04401088, 03624134, 04004475
  warnings.warn(msg)


['airplane', 'trash bin', 'bag', 'basket', 'bathtub', 'bed', 'bench', 'birdhouse', 'bookshelf', 'bottle', 'bowl', 'bus', 'cabinet', 'camera', 'can', 'cap', 'car', 'cellphone', 'chair', 'clock', 'keyboard', 'dishwasher', 'display', 'earphone', 'faucet', 'file cabinet', 'guitar', 'helmet', 'jar', 'knife', 'lamp', 'laptop', 'loudspeaker', 'mailbox', 'microphone', 'microwaves', 'motorbike', 'mug', 'piano', 'pillow', 'pistol', 'flowerpot', 'printer', 'remote', 'rifle', 'rocket', 'skateboard', 'sofa', 'stove', 'table', 'telephone', 'tower', 'train', 'watercraft', 'washer']


In [5]:
shapenet_dataset[0]

{'synset_id': '04530566',
 'model_id': '6a0da87e5858b4373e45dd2f173cbf9b',
 'verts': tensor([[ 0.0616, -0.0442, -0.1398],
         [ 0.0616, -0.0447, -0.1399],
         [ 0.0616, -0.0442, -0.1399],
         ...,
         [-0.0014, -0.1220,  0.0110],
         [-0.0013, -0.1220,  0.0111],
         [-0.0179,  0.0785, -0.2876]]),
 'faces': tensor([[    0,     1,     2],
         [    1,     0,     3],
         [    2,     1,     0],
         ...,
         [73306, 73307, 73310],
         [73307, 73310, 73309],
         [73309, 73310, 73307]]),
 'textures': None,
 'label': 'watercraft'}

In [6]:
import plotly.io as pio
print(pio.renderers)
pio.renderers.default = "browser"

Renderers configuration
-----------------------
    Default renderer: 'plotly_mimetype+notebook'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [7]:
import plotly.graph_objects as go

def render_mesh(mesh):
    verts, faces = mesh['verts'], mesh['faces']
    # Render the mesh using plotly
    fig = go.Figure(
        data=[go.Mesh3d(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
                        i=faces[:, 0], j=faces[:, 1], k=faces[:, 2], # opacity=0.5,
                        # lightposition=dict(x=0, y=0, z=0.2)
                        )
              ],)
    # Keep the aspect ratio of the mesh
    fig.update_layout(scene_aspectmode='data')
    # Set dark mode
    fig.update_layout(template='plotly_dark')
    fig.show()

for i in range(5):
    render_mesh(shapenet_dataset[i])

In [ ]:
from pathlib import Path

def get_sdf_grid_from_mesh(grid_points, mesh):
    signed_distances, face_ids, barycentric_coords = pcu.signed_distance_to_mesh(grid_points, mesh['verts'], mesh['faces'], lower_bound=-100, upper_bound=1000)

    return signed_distances

def watertight_mesh(mesh):
    verts, faces = pcu.make_mesh_watertight(mesh['verts'], mesh['faces'], resolution=20_000)
    return {'verts': verts, 'faces': faces}

